In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

機器人被加入群組時觸發

向群組說聲早安

"""


from linebot.models import (
    JoinEvent, TextSendMessage,
)

@handler.add(JoinEvent)
def handle_join(event):
    print(event)
    
    print("群組 ID：" + event.source.group_id)
    
    reply_message_list = [
        TextSendMessage(text="大家早 ~\n我是來自火星的機器人，很榮幸加入各位的群組！"),
        TextSendMessage(text="題外話，你們知道每個群組都有一個 ID 嗎？"),
        TextSendMessage(text="我絕對不會告訴各位，你們的群組 ID 是 " + event.source.group_id)
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [ ]:
"""

人類 被加入群組時觸發

機器人 向新加入的人類問好

"""

from linebot.models import (
    MemberJoinedEvent, TextSendMessage, ImageSendMessage
)

import json

@handler.add(MemberJoinedEvent)
def handle_member_join(event):
    print(event)
    
    group_id = event.source.group_id
    joined_members_list = event.joined.members
    
    members_name_list = []
    for member in joined_members_list:
        
        # 從 members 串列 中一個一個取出 member
        # 並利用取出的 member user id 與群組 id，向 LINE 官方取得用戶個資
        user_id = member.user_id
        profile = line_bot_api.get_group_member_profile(group_id, user_id)
        
        # 將用戶個資存檔
        with open('user_in_the_group.json', 'a', encoding='utf-8') as f:
            user_profile_dict = profile.__dict__
            user_profile_dict['group_id'] = group_id
            f.write(json.dumps(user_profile_dict))
            f.write('\n')
        
        # 準備用戶名字串列，等等用來打招呼
        members_name_list.append(profile.display_name)

    members_name_str = ", ".join(members_name_list)
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=members_name_str + " 你(們)好\n我是這個群組唯一的機器人，請多多指教！")
    )

In [ ]:
# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 



## 點擊後，以Postback事件回應Server 
a_dream_of_red_mansions = QuickReplyButton(
    action=PostbackAction(label="紅樓夢", data="book=紅樓夢")
)

outlaws_of_the_marsh = QuickReplyButton(
    action=PostbackAction(label="水滸傳", data="book=水滸傳")
)


the_journey_to_the_west = QuickReplyButton(
    action=PostbackAction(label="西遊記", data="book=西遊記")
)

romance_of_three_kingdoms = QuickReplyButton(
    action=PostbackAction(label="三國演義", data="book=三國演義")
)

'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [a_dream_of_red_mansions, outlaws_of_the_marsh, the_journey_to_the_west, romance_of_three_kingdoms]
)

In [ ]:
"""

當群組中有人說話時，會觸發機器人

這部分的動作跟單人版是一樣的

"""

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, PostbackEvent
)


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print(event)
    
    if "推薦" in event.message.text:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="請選擇想了解的書籍", quick_reply=quickReplyList)
        )
    
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="很高興為您服務！")
        )


In [ ]:
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    print(event)
    
    query_string_dict = parse_qs(event.postback.data)
    print(query_string_dict)
    
    if 'book' in query_string_dict:
        book_name = query_string_dict.get('book')[0]
        
        reply_dict = {
            "三國演義": TextSendMessage(text="《三國演義》描寫的是從東漢末年到西晉初年近百年間的歷史，反映了三國時代的政治軍事鬥爭以及各類社會矛盾的滲透與轉化。在對三國態度上，尊劉反曹鄙吳是民間的主要傾向，以劉備集團作為描寫的中心，隱含著人民對漢室復興的希望和皇室正統思想，儘管這些舊有觀點已不容於今日。"),
            "水滸傳": TextSendMessage(text="水滸中的一百零八將，是張天師禁錮的三十六個天罡星和七十二個地煞星，因為太尉洪信的無心之過而轉世，他們講究義氣，愛打抱不平、劫富濟貧，不滿貪官污吏，最後集結梁山，與腐化的朝廷抗爭。小說成功地塑造了宋江、林冲、史進、李逵、魯智深、武松等人物的鮮明形象，也向讀者展示了宋代的政治與社會狀況。"),
            "紅樓夢": TextSendMessage(text="作者以貴族家庭的興衰為主軸。為避免文字觸及時事，躲避清政府文字獄，於是虛構朝代，隱去真事，並以描寫閨閣女子來避諱。不過此書最為人所熟知的是賈寶玉和林黛玉的愛情及二人與薛寶釵之間的關係。"),
            "西遊記": TextSendMessage(text="西遊記的主角五人（唐僧師徒四人和白龍馬）對應著五行：金對應著孫悟空，又稱「金公」、「心猿」；木對應著豬八戒，又稱「木母」、「木龍」；土對應著沙僧，又稱「刀圭」、「黃婆」，水、火對應何人書中並未明言，一般認為唐僧對應水（因為「江流兒」），白龍馬對應火；也有人認為唐僧對應火（因火能克金），白龍馬對應水。白龍馬又稱「意馬」，與孫悟空合成「心猿意馬」之意。"),
        }
        
        line_bot_api.reply_message(
            event.reply_token,
            reply_dict.get(book_name)
        )

In [ ]:
'''

設定一個Server入口，

當有人訪問此入口，會對群組進行消息推播

請注意一次推播用掉的訊息額度，等於群組中的用戶數

'''

from linebot.models import (
    FlexSendMessage
)

import json

@app.route("/pushMessage", methods=['GET'])
def push_message():
    
    # 載入訊息設定檔
    with open('flex_message.json', 'r', encoding='utf-8') as f:
        message_dict = json.load(f)
    
    buttons_template_message = FlexSendMessage.new_from_json_dict(message_dict)
    
    # 推送消息至群組，須先取得群組 ID
    line_bot_api.push_message(
        'C886eb415bbfec9a1282ef54463aecf55',
        buttons_template_message
    )

    return '消息已發送'

In [ ]:

'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')